# Email Classification

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('./datasets/email_spam.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2636 entries, 0 to 2635
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Kategori  2636 non-null   object
 1   Pesan     2636 non-null   object
dtypes: object(2)
memory usage: 41.3+ KB


In [40]:
df.sample(5)

,Kategori,Pesan
1464,ham,Kunjungan Anda dengan Vince Kaminski Enron Co...
537,spam,quiicker halo selamat datang di juri pha rmonl...
1478,ham,Re Kunjungan berikutnya ke Houston George apak...
2233,ham,Pengumuman Organisasi Memperkenalkan Layanan ...
816,spam,di jantung bisnis Anda Citra perusahaan dapat ...


In [ ]:
stopwords = pd.read_csv('datasets/stopwords.csv', header=None, names=['words'])
stopwords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 758 entries, 0 to 757
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   words   758 non-null    object
dtypes: object(1)
memory usage: 6.1+ KB


In [41]:
stopwords.sample(5)

,words
591,selain
628,sepihak
676,tampaknya
580,sekalipun
488,para


## Preprocessing

### Membersihkan data dari yang bukan teks (angka, tanda baca)

In [39]:
import re
import string

clean_text = lambda text: re.sub(f'[{string.punctuation}0-9]', '', text)
df['Pesan'] = df['Pesan'].apply(clean_text)

df.sample(5)

,Kategori,Pesan
1994,ham,Re FW Citi Wells Enron SL dan I membentuk B ...
453,spam,Amazingg halo selamat datang di ph armonline y...
2444,ham,Re Senang bertemu Anda di pertemuan Informs Sa...
2016,ham,Re Anita DuPont melanjutkan Sheila tidak kita ...
549,spam,Perangkat Lunak Perbandingan Kebijakan LTCI GR...
1185,spam,Hidup tahun lebih lama dengan HGH Halo ke ...
2002,ham,pembelian lisensi untuk perangkat lunak halo s...
1714,ham,Aplikasi Yvan Molly saya melampirkan surat rek...
1764,ham,Maddox Vince di sini adalah Informasi Kontak M...
1085,spam,Perdagangan untuk mencari nafkah semua yang ha...


### Tokenisasi

In [43]:
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize, sent_tokenize

# Contoh teks
text = "Halo! Ini adalah contoh tokenisasi. Bagaimana kabar kamu?"

# Tokenisasi kata
word_tokens = word_tokenize(text)
print("Token Kata:", word_tokens)

# Tokenisasi kalimat
sentence_tokens = sent_tokenize(text)
print("Token Kalimat:", sentence_tokens)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\munaf\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\munaf/nltk_data'
    - 'c:\\Users\\munaf\\AppData\\Local\\Programs\\Python\\Python312\\nltk_data'
    - 'c:\\Users\\munaf\\AppData\\Local\\Programs\\Python\\Python312\\share\\nltk_data'
    - 'c:\\Users\\munaf\\AppData\\Local\\Programs\\Python\\Python312\\lib\\nltk_data'
    - 'C:\\Users\\munaf\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [8]:
df.isnull().sum()

Kategori    0
Pesan       0
dtype: int64

In [11]:
df.groupby('Kategori').count()

,Pesan
Kategori,
ham,1268
spam,1368
